<a href="https://colab.research.google.com/github/kiruthihan10/smile-Recognition-with-Tensorflow/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from PIL import Image
import random


In [2]:
height = 64
width = 64
channels = 3
n_inputs = height * width

conv1_fmaps = 96
conv1_ksize = 16
conv1_stride = 4
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 4
conv2_stride = 4
conv2_pad = "SAME"


pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 2


with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height,width,channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 2 * 2])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def img_loader():
    common_path = "/content/gdrive/My Drive/Smile_Recognition/faces/"
    txt_path = "/content/gdrive/My Drive/Smile_Recognition/"
    id_add = open(txt_path+"ALL_LIST_ppm.txt",'r')
    print("Txtfile Opened!!!")
    images = []
    for line in id_add:
      #print(line.strip())
      if True:
        grand_row = []
        img = Image.open(common_path+line.strip())
        img= (np.array(img))
        images.append(img)
    #print(len(grand_row))
    return images

def smile_check():
    txt_path = "/content/gdrive/My Drive/Smile_Recognition/"
    smile = open(txt_path+"SMILE_list_ppm.txt",'r')
    print("Smile List Opened")
    id_add = open(txt_path+"ALL_LIST_ppm.txt",'r')
    print("All list opened")
    smile_list = []
    for line in smile:
        smile_list.append(line.strip())
    smiles = []
    x=0
    for name in id_add:
        if name.strip() in smile_list:
            smiles.append(True)
            #print (x)
        else:
            smiles.append(False)
        x+=1
    #print(len(smile_list))
    smile.close()
    id_add.close()
    return smiles

In [5]:
dataset = np.array(img_loader(),dtype=np.float32)
batch_size,height,width,channels = dataset.shape
print(dataset.shape)

Txtfile Opened!!!
(1203, 64, 64, 3)


In [6]:
output = smile_check()
temp = []

for i in output:
  if i == True:
    temp.append(1)
  else:
    temp.append(0)
output = temp
output=np.array(output,dtype=bool)
output.shape

Smile List Opened
All list opened


(1203,)

In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
def shuffle2list(x,y):
  mapIndexPosition = list(zip(x,y))
  random.shuffle(mapIndexPosition)
  x,y = zip(*mapIndexPosition)
  return x,y

In [0]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [10]:
dataset,output = shuffle_in_unison(dataset,output)
print(dataset.shape)
X_train = np.array(dataset[:1000])
y_train = np.array(output[:1000])
X_test = np.array(dataset[1000:,:,:,:])
y_test = np.array(output[1000:])
print(X_train.shape)
print(X_test.shape)

(1203, 64, 64, 3)
(1000, 64, 64, 3)
(203, 64, 64, 3)


In [11]:
n_epochs = 50
batch_size = 25

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        X_batch,y_batch=shuffle2list(X_train,y_train)
        index = 0
        while index+batch_size<len(X_batch):
            sess.run(training_op, feed_dict={X: X_batch[index:index+batch_size], y: y_batch[index:index+batch_size]})
            index+=batch_size
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")

0 Last batch accuracy: 0.52 Test accuracy: 0.4827586
1 Last batch accuracy: 0.789 Test accuracy: 0.81280786
2 Last batch accuracy: 0.834 Test accuracy: 0.85714287
3 Last batch accuracy: 0.892 Test accuracy: 0.8719212
4 Last batch accuracy: 0.9 Test accuracy: 0.87684727
5 Last batch accuracy: 0.879 Test accuracy: 0.8374384
6 Last batch accuracy: 0.925 Test accuracy: 0.9014778
7 Last batch accuracy: 0.854 Test accuracy: 0.8522167
8 Last batch accuracy: 0.894 Test accuracy: 0.84729064
9 Last batch accuracy: 0.943 Test accuracy: 0.8965517
10 Last batch accuracy: 0.948 Test accuracy: 0.8817734
11 Last batch accuracy: 0.927 Test accuracy: 0.8866995
12 Last batch accuracy: 0.957 Test accuracy: 0.8965517
13 Last batch accuracy: 0.966 Test accuracy: 0.8965517
14 Last batch accuracy: 0.967 Test accuracy: 0.90640396
15 Last batch accuracy: 0.913 Test accuracy: 0.8325123
16 Last batch accuracy: 0.952 Test accuracy: 0.87684727
17 Last batch accuracy: 0.962 Test accuracy: 0.8965517
18 Last batch acc